# Useful ADS Bits and pieces that made my life.. easy ish



# <u>Snippets</U>  (Extension)
>Quickly create re-useable code in simple steps
1. Open New SQL query window
2. Ctrl-Shift+P
3. Write some SQL you want to save
4. Type QE Boost and select QE Boost: Save New snippet
5. Give it a name starting SQL
6. Enter description
7. Then just Type in any query window sql.....name of the snippet
>

In [ ]:
IF (SELECT OBJECT_ID('Tempdb..#DatabaseAccess')) IS NOT NULL DROP TABLE #DatabaseAccess
CREATE TABLE #DatabaseAccess
(
     ServerName VARCHAR(50)
    ,DatabaseName VARCHAR(100)
    ,ReportDate DATETIME
    ,user_seeks BIGINT
    ,user_scans BIGINT
    ,user_lookups BIGINT
    ,reads BIGINT
    ,writes BIGINT
    ,ServerStartDate DATETIME

)

 
EXEC sp_MSFOREACHDB 'USE [?]

IF db_name() NOT IN (''TempDB'',''model'',''msdb'',''DBATools'',''master'')

INSERT INTO #DatabaseAccess
(
     ServerName
    ,DatabaseName
    ,ReportDate
    ,user_seeks
    ,user_scans
    ,user_lookups
    ,reads
    ,writes
    ,ServerStartDate

)


SELECT      
    @@ServerName AS ServerName,
    db_name() AS DatabaseName,
    CAST(GETDATE() as DATETIME) as ReportDate,
    SUM(user_seeks) AS user_seeks,   
    SUM(user_scans) AS user_scans,
    SUM(user_lookups) AS user_scans,
    reads=SUM(user_seeks + user_scans + user_lookups),         
    writes =  SUM(user_updates),   
    ServerStartDate = (select login_time from master.dbo.sysprocesses where spid =1)
FROM 
    sys.dm_db_index_usage_stats s        
INNER JOIN       
    sys.indexes i ON i.index_id = s.index_id AND s.object_id = i.object_id         
INNER JOIN       
    sys.objects o on s.object_id = o.object_id      
INNER JOIN       
    sys.schemas c on o.schema_id = c.schema_id  
INNER JOIN    
     sys.partitions AS p ON p.OBJECT_ID = i.OBJECT_ID AND p.index_id = i.index_id  
INNER JOIN   
     sys.allocation_units AS a ON a.container_id = p.partition_id  
WHERE       
        (OBJECTPROPERTY(s.object_id,''IsUserTable'') = 1  or OBJECTPROPERTY(s.object_id,''IsView'') = 1)
     AND       
        i.name IS NOT NULL AND s.database_id = DB_ID() 
'	

SELECT 
    ServerName
    ,DatabaseName
    ,ReportDate
    ,COALESCE(user_seeks,0) AS user_seeks
    ,COALESCE(user_scans,0) AS user_scans
    ,COALESCE(user_lookups,0) AS user_lookups
    ,COALESCE(reads,0) AS reads
    ,COALESCE(writes,0) AS writes
    ,ServerStartDate
 FROM #DatabaseAccess




# Some more useful snippets

> ## Index stats

In [ ]:
IF (SELECT OBJECT_ID('Tempdb..#DatabaseAccess')) IS NOT NULL DROP TABLE #DatabaseAccess
CREATE TABLE #DatabaseAccess
(
     ServerName VARCHAR(50)
    ,DatabaseName VARCHAR(100)
    ,ReportDate DATETIME
    ,user_seeks BIGINT
    ,user_scans BIGINT
    ,user_lookups BIGINT
    ,reads BIGINT
    ,writes BIGINT
    ,ServerStartDate DATETIME

)

 
EXEC sp_MSFOREACHDB 'USE [?]

IF db_name() NOT IN (''TempDB'',''model'',''msdb'',''DBATools'',''master'')

INSERT INTO #DatabaseAccess
(
     ServerName
    ,DatabaseName
    ,ReportDate
    ,user_seeks
    ,user_scans
    ,user_lookups
    ,reads
    ,writes
    ,ServerStartDate

)


SELECT      
    @@ServerName AS ServerName,
    db_name() AS DatabaseName,
    CAST(GETDATE() as DATETIME) as ReportDate,
    SUM(user_seeks) AS user_seeks,   
    SUM(user_scans) AS user_scans,
    SUM(user_lookups) AS user_scans,
    reads=SUM(user_seeks + user_scans + user_lookups),         
    writes =  SUM(user_updates),   
    ServerStartDate = (select login_time from master.dbo.sysprocesses where spid =1)
FROM 
    sys.dm_db_index_usage_stats s        
INNER JOIN       
    sys.indexes i ON i.index_id = s.index_id AND s.object_id = i.object_id         
INNER JOIN       
    sys.objects o on s.object_id = o.object_id      
INNER JOIN       
    sys.schemas c on o.schema_id = c.schema_id  
INNER JOIN    
     sys.partitions AS p ON p.OBJECT_ID = i.OBJECT_ID AND p.index_id = i.index_id  
INNER JOIN   
     sys.allocation_units AS a ON a.container_id = p.partition_id  
WHERE       
        (OBJECTPROPERTY(s.object_id,''IsUserTable'') = 1  or OBJECTPROPERTY(s.object_id,''IsView'') = 1)
     AND       
        i.name IS NOT NULL AND s.database_id = DB_ID() 
'	

SELECT 
    ServerName
    ,DatabaseName
    ,ReportDate
    ,COALESCE(user_seeks,0) AS user_seeks
    ,COALESCE(user_scans,0) AS user_scans
    ,COALESCE(user_lookups,0) AS user_lookups
    ,COALESCE(reads,0) AS reads
    ,COALESCE(writes,0) AS writes
    ,ServerStartDate
 FROM #DatabaseAccess




# Job steps snippet
> ## Easy way to see Jobsteps with blank lines added to aid clarity

In [ ]:

WITH CTE_JobSteps
AS
(
SELECT sj.Name,sj.[description],sjs.step_name,sjs.step_id,sjs.command,sjs.subsystem,sjs.database_name, ROW_NUMBER() OVER (Partition by sj.Name  Order By sj.Name,sjs.step_id ) AS RowID, 1 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
UNION ALL
SELECT sj.Name ,'' AS description,'' AS step_name,'' AS step_id,'' AS command,'' AS subsystem,'' AS database_name,MAX(step_id)+1 AS RowID,0 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
GROUP BY sj.name
)
SELECT 
    CASE WHEN show =1 THEN  Name ELSE '' END AS Name
    ,CASE WHEN show =1 THEN  CAST(Step_id AS CHAR(4)) ELSE '' END AS Step_id
    ,step_name
    ,command
    ,subsystem
    ,database_name
FROM CTE_JobSteps c
ORDER BY c.Name,c.RowID

## Converting sp_execute 
> ### (Shift+Ctrl+P) and search for convert

In [ ]:
DECLARE @P1 varchar(50),@P2 int,@P3 int,@P4 varchar(50),@P5 bit,@P6 bit,@P7 bit,@P8 bit,@P9 bit,@P10 bit,@P11 bit,@P12 bit,@P13 nvarchar(50),@P14 int,@P15 int,@P16 nvarchar(1),@P17 bit,@P18 int,@P19 datetime,@P20 bit,@P21 bit,@P22 nvarchar(15),@P23 binary(250),@P24 int,@P25 datetime
SET @P1='1234'
SET @P2=666
SET @P3=514
SET @P4='SD'
SET @P5=0
SET @P6=0
SET @P7=0
SET @P8=0
SET @P9=0
SET @P10=0
SET @P11=0
SET @P12=0
SET @P13=N'11152930'
SET @P14=10
SET @P15=120
SET @P16=N'A'
SET @P17=0
SET @P18=2100
SET @P19='2022-04-18 10:29:22.390'
SET @P20=0
SET @P21=1
SET @P22=N'00000000'
SET @P23=0x010500000000005000000833DXX61199B978828B85E0E00000000000000000000000000
SET @P24=120
SET @P25='2001-12-14 00:00:00'

UPDATE "dbo"."ExampleTable" SET "PassportNumber"=@P1 WHERE "Emp_No" = @P2 AND "PayrollID" = @P3 AND "UKNO" = @P4 AND "PassportNumber" IS NULL AND "Time1" IS NULL AND "Time2" IS NULL AND "Time1_FE" IS NULL AND "Time2_FE" IS NULL AND "Time1_Rate" IS NULL AND "Time2_Rate" IS NULL AND "Phone_Book_ID" IS NULL AND "Hide_From_PR" = @P5 AND "Hide_From_Grid" = @P6 AND "Requires_PC" = @P7 AND "Spare_2" = @P8 AND "Spare_3"
= @P9 AND "Spare_4" = @P10 AND "Spare_5" = @P11 AND "Spare_6" = @P12 AND "Employee_Ref" IS NULL AND "NI_NO" IS NULL AND "Payroll_Number" IS NULL AND "Passport_1_No" = @P13 AND "Passport_1_Country" = @P14 AND "Passport_2_No" IS NULL AND "Country" = @P15 AND "Passport_3_No" IS NULL AND "Passport_3_Country" IS NULL AND "DACEE_ID" IS NULL AND "Bel_Nat_No" IS NULL AND "Bel_Payroll_Number" IS NULL AND "Salary" IS NULL AND "Currency" IS NULL AND "NI_Code" = @P16 AND "LWB" = @P17 AND "_LWB_Changed_By" = @P18 AND "_LWB_Changed_Date" = @P19 AND "XY" = @P20 AND "Partners_Address_Book" = @P21 AND "ABC" IS NULL AND "PRT" IS NULL AND "SAP_ID" = @P22 AND "SID" = @P23 AND "X_Code" IS NULL AND "VAT_Code" IS NULL AND "CCMX" IS NULL AND "Country_of_birth" = @P24 AND "ExpiryDate" = @P25 AND "ExpiryDate2" IS NULL AND "ExpiryDate3" IS NULL


# Formatting SQL
>## ctrl+shift+ P type poor SQL or set a hotkey (Shift-I) (Extension)

In [ ]:
-- List columns in all tables whose name is like 'TableName'
SELECT TableName = tbl.table_schema + '.' + tbl.table_name
    , ColumnName = col.column_name
    , ColumnDataType = col.data_type
FROM INFORMATION_SCHEMA.TABLES tbl
INNER JOIN INFORMATION_SCHEMA.COLUMNS col
    ON col.table_name = tbl.table_name
        AND col.table_schema = tbl.table_schema
WHERE tbl.table_type = 'base table'
    AND tbl.table_name LIKE '%TableName%'
GO

SELECT e.[BusinessEntityID]
    , p.[Title]
    , p.[FirstName]
    , p.[MiddleName]
    , p.[LastName]
    , p.[Suffix]
    , e.[JobTitle]
    , pp.[PhoneNumber]
    , pnt.[Name] AS [PhoneNumberType]
    , ea.[EmailAddress]
    , p.[EmailPromotion]
    , a.[AddressLine1]
    , a.[AddressLine2]
    , a.[City]
    , sp.[Name] AS [StateProvinceName]
    , a.[PostalCode]
    , cr.[Name] AS [CountryRegionName]
    , p.[AdditionalContactInfo]
FROM [HumanResources].[Employee] e
INNER JOIN [Person].[Person] p
    ON p.[BusinessEntityID] = e.[BusinessEntityID]
INNER JOIN [Person].[BusinessEntityAddress] bea
    ON bea.[BusinessEntityID] = e.[BusinessEntityID]
INNER JOIN [Person].[Address] a
    ON a.[AddressID] = bea.[AddressID]
INNER JOIN [Person].[StateProvince] sp
    ON sp.[StateProvinceID] = a.[StateProvinceID]
INNER JOIN [Person].[CountryRegion] cr
    ON cr.[CountryRegionCode] = sp.[CountryRegionCode]
LEFT OUTER JOIN [Person].[PersonPhone] pp
    ON pp.BusinessEntityID = p.[BusinessEntityID]
LEFT OUTER JOIN [Person].[PhoneNumberType] pnt
    ON pp.[PhoneNumberTypeID] = pnt.[PhoneNumberTypeID]
LEFT OUTER JOIN [Person].[EmailAddress] ea
    ON p.[BusinessEntityID] = ea.[BusinessEntityID];
GO




# Generating SQL Scripts
> ## Ability to script a table and specify where clause (extention)
1. Install extension
2. Go to server and drillk through to table and right click select script as

# Running a Query you've aready ran
> ## Ability execute previously ran code (extention)
1. Install extension
2. Type ctrl-j and select Query History